# Coupon optimization model using PULP

Linear optimization is a method to optimize problems (maximize or minimize a function) given some constraints. In this notebook, we will use an example of 3 "fake" users who are candidate to receive a coupon (ranging from 2 to 10). To perform the optimization, we need two inputs:

* Probability of accepting (conversion probability) for every possible coupon. While in this example we will simulate "fake" but consistent probabilities, in the real life this probabilities will be obtained from a ML model.
* Expected return of the user. In this example, we will invent the expected return of the users, but in the real life this expected return will come from a common businees metric such as the LTV.

For optimizing, there are two main components we need to keep in mind:

* Objective: Function to optimize. In this case, we will maximize the revenue given a certain coupon n:

    $ revenue = p(conversion | cn) * (LTV - cn)$

    where $p(conversion | cn)$ is the probability of accepting coupon n (or conversion probability given coupon n), $LTV$ is the user's expected return, and $cn$ is the coupon value.

* Constraints: They are the restrictions given by the problem's design. The constrains are used to obtain the result within we are expecting and for limiting the searchin space of the problem.

    * One coupon per user
    * $\sum c_i <= b$: Sum of coupons cannot exceed the budget (b)

In [1]:
import numpy as np
import datetime
import pulp as plp
import pandas as pd
from random import random

In [2]:
col_prob = ['coupon_{}'.format(i) for i in range(2,11,1)] ## define the prob_ columns to the range of the coupons desired, in this 
# target matrix

## Creating the decision dataframe: this is the df that we are expecting from the optimizer: 
##    A flag (1 or 0) indicating if the user receives the coupon or not
prob = pd.DataFrame(columns= col_prob)
arr_zeros = np.zeros((1,9)).astype('int')
arr_zeros[0,int(random()*9)] = 1
prob.loc[1,col_prob] = np.array(arr_zeros)
arr_zeros = np.zeros((1,9)).astype('int')
arr_zeros[0,int(random()*9)] = 1
prob.loc[2,col_prob] = np.array(arr_zeros)
arr_zeros = np.zeros((1,9)).astype('int')
arr_zeros[0,int(random()*9)] = 1
prob.loc[3,col_prob] = np.array(arr_zeros)

prob.head()

,coupon_2,coupon_3,coupon_4,coupon_5,coupon_6,coupon_7,coupon_8,coupon_9,coupon_10
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Simple optimization experiment for 3 users

In [3]:
col_prob = ['prob_{}'.format(i) for i in range(2,11,1)] ## define the prob_ columns to the range of the coupons desired, in this 
# scenario the range is from 2 to 10
factor3 = 0.03 #define the prob multiplicative factors that will show how the probabilities will change for each user given that will increment their coupon based on the range
factor2 = 0.02 #define the prob multiplicative factors that will show how the probabilities will change for each user given that will increment their coupon based on the range
factor1 = 0.08 #define the prob multiplicative factors that will show how the probabilities will change for each user given that will increment their coupon based on the range
ltv1 = 50 #define "fake"LTV for client 1 
ltv2 = 70 #define "fake"LTV for client 2
ltv3 = 100 #define "fake"LTV for client 3
col_prob_1 = [i*factor1 for i in range(2,11,1)] #define how the probability will change with higher coupons 
col_prob_2 = [i*factor2 for i in range(2,11,1)] #define how the probability will change with higher coupons
col_prob_3 = [i*factor3 for i in range(2,11,1)] #define how the probability will change with higher coupons
col_prob_1.append(ltv1)
col_prob_2.append(ltv2)
col_prob_3.append(ltv3)

## Creating the dataframe
prob = pd.DataFrame(columns= col_prob+['6m_REVENEU'])
prob.loc[1,col_prob+['6m_REVENEU']] = np.array(col_prob_1)
prob.loc[2,col_prob+['6m_REVENEU']] = np.array(col_prob_2)
prob.loc[3,col_prob+['6m_REVENEU']] =  np.array(col_prob_3)

prob.head()

,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,prob_9,prob_10,6m_REVENEU
1,0.16,0.24,0.32,0.4,0.48,0.56,0.64,0.72,0.8,50
2,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,70
3,0.06,0.09,0.12,0.15,0.18,0.21,0.24,0.27,0.3,100


In [4]:
## Revenue expected for "Business rules" of coupon based on 10% of the LTV
rev_bau = prob.loc[1,'prob_{}'.format(int(ltv1/10))]*(ltv1*0.9)+prob.loc[2,'prob_{}'.format(int(ltv2/10))]*(ltv2*0.9)+prob.loc[3,'prob_{}'.format(int(ltv3/10))]*(ltv3*0.9)
rev_bau

53.82

## Defining the Optimization Problem

In [5]:
set_I = [1,2,3] ## total users
coupon_range = range(2,11,1) # coupon range
display(coupon_range)
coupons_per_contract = {i: coupon_range for i in set_I} ## dictionary of coupons avaliable to each user
display(coupons_per_contract)

range(2, 11)

{1: range(2, 11), 2: range(2, 11), 3: range(2, 11)}

In [6]:
# Problema definition
#A is the matrix of the coupons, 
#X is the matrix that selects only one coupon per user (basically we ensure that each row of this matrix sum to 1 and that is a binary variable)
#b is our budget constrain 
#c is our probability matrix
#d is our ltv-cost matrix
print('\tCreate matrices max(cx) st Ax=>b and Ax<=b ; sum(x_j) = 1\n')

x_vars = dict()
a = dict()
c = dict()
d = dict()

for k, i in enumerate(set_I):
    for j in coupons_per_contract[i]:
       

        x_vars[(i, j)] = plp.LpVariable(cat=plp.LpInteger, lowBound=0,
                                        upBound=1, name="x_{0}_{1}".format(i, j)) ## integer matrix where the rows sum to one, selecting only one coupon per user

        a[(i, j)] = j # coupon matrix

        c[(i, j)] = prob.loc[i]['prob_{}'.format(j)]  # probability matrix

        d[(i,j)] = prob.loc[i]['6m_REVENEU'] - j  #ltv - cost matrix

	Create matrices max(cx) st Ax=>b and Ax<=b ; sum(x_j) = 1



### Constraints definition

In [7]:
opt_model = plp.LpProblem(name="MIP Model - LTV - COST maximization \n") ## name of the model

constraints = {i:
               plp.LpConstraint(
                   e=plp.lpSum(x_vars[i, j]
                               for j in coupons_per_contract[i]), 
                   sense=plp.LpConstraintEQ,
                   rhs=1*(len(coupons_per_contract[i]) > 0), ## making sure that only one coupon to each client if the client is elegible to a coupon
                   name="constraint_{0}".format(i))
               for i in set_I}

b = (ltv1+ltv2+ltv3)/10 ## total budget is the same as for the business rules (10% of the LTV)

print('\t Restriction 2 : {} ...\n'.format(b))

constraints[-1] = plp.LpConstraint(
    e=plp.lpSum(a[i, j] * x_vars[i, j] for i in set_I for j in coupons_per_contract[i]), 
    sense=plp.LpConstraintLE,
    rhs=b,
    name="constraint_-2") ## making sure that the coupons don't exceed the budget
constraints ## all contrains

	 Restriction 2 : 22.0 ...



/home/wilomaku/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


{1: 1*x_1_10 + 1*x_1_2 + 1*x_1_3 + 1*x_1_4 + 1*x_1_5 + 1*x_1_6 + 1*x_1_7 + 1*x_1_8 + 1*x_1_9 + -1 = 0,
 2: 1*x_2_10 + 1*x_2_2 + 1*x_2_3 + 1*x_2_4 + 1*x_2_5 + 1*x_2_6 + 1*x_2_7 + 1*x_2_8 + 1*x_2_9 + -1 = 0,
 3: 1*x_3_10 + 1*x_3_2 + 1*x_3_3 + 1*x_3_4 + 1*x_3_5 + 1*x_3_6 + 1*x_3_7 + 1*x_3_8 + 1*x_3_9 + -1 = 0,
 -1: 10*x_1_10 + 2*x_1_2 + 3*x_1_3 + 4*x_1_4 + 5*x_1_5 + 6*x_1_6 + 7*x_1_7 + 8*x_1_8 + 9*x_1_9 + 10*x_2_10 + 2*x_2_2 + 3*x_2_3 + 4*x_2_4 + 5*x_2_5 + 6*x_2_6 + 7*x_2_7 + 8*x_2_8 + 9*x_2_9 + 10*x_3_10 + 2*x_3_2 + 3*x_3_3 + 4*x_3_4 + 5*x_3_5 + 6*x_3_6 + 7*x_3_7 + 8*x_3_8 + 9*x_3_9 + -22.0 <= 0}

### Objective definition

In [8]:
df = pd.DataFrame(columns=['coupon_optimal','p_coupon_optimal'])
objective = plp.lpSum(x_vars[i, j] * c[i, j] * d[i,j]
                      for i in set_I for j in coupons_per_contract[i])
# our model objective maximazing expected LTV - cost 
# matrix d (LTV - cost)* matrix c (prob_matrix) * matrix x (matrix of selecting only on coupon per client)

for k in constraints.keys():
    opt_model.addConstraint(constraints[k])

opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

### Solving

In [9]:
#opt_model.solve(plp.solvers.COIN(fracGap=TOLERANCIA, msg=0))
#opt_model.solve(plp.solvers.PULP_CBC_CMD(fracGap=TOLERANCIA, msg=0))
opt_model.solve() ## solve the model

print('\tProblem {}!\n'.format(plp.constants.LpStatus[opt_model.status])) ## status

print('\tPrepare coupon data\n')
sel_coupon = pd.Series([])
for var in opt_model.variables():

    if var.varValue == 1:
        i, j = int(var.name.split('_')[1]), int(var.name.split('_')[2]) ##saving the optimal results
       

        sel_coupon[i] = {'AMOUNT_RC': j,
                                  'p': prob.loc[i]['prob_{}'.format(j)],
                                 }

	Problem Optimal!

	Prepare coupon data



<ipython-input-9-4fb78e5d2fd1>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sel_coupon = pd.Series([])


In [10]:
sel_coupon

1    {'AMOUNT_RC': 10, 'p': 0.8}
2    {'AMOUNT_RC': 2, 'p': 0.04}
3    {'AMOUNT_RC': 10, 'p': 0.3}
dtype: object

In [11]:
rev_model = opt_model.objective.value() ## evaluating optimail solution
rev_model

61.72

In [12]:
rev_model > rev_bau
#Optimal solution is better than the business rules!!!

True

## Testing another solver

In [13]:
#opt_model = plp.getSolver('CPLEX_CMD')
opt_model = plp.LpProblem(name="MIP Model - LTV - COST maximization \n") ## name of the model

constraints = {i:
               plp.LpConstraint(
                   e=plp.lpSum(x_vars[i, j]
                               for j in coupons_per_contract[i]), 
                   sense=plp.LpConstraintEQ,
                   rhs=1*(len(coupons_per_contract[i]) > 0), ## making sure that only one coupon to each client if the client is elegible to a coupon
                   name="constraint_{0}".format(i))
               for i in set_I}

b = (ltv1+ltv2+ltv3)/10 ## total budget is the same as for the business rules (10% of the LTV)
print('\t Restriction 2 : {} ...\n'.format(b))
constraints[-1] = plp.LpConstraint(
    e=plp.lpSum(a[i, j] * x_vars[i, j] for i in set_I for j in coupons_per_contract[i]), 
    sense=plp.LpConstraintLE,
    rhs=b,
    name="constraint_-2") ## making sure that the coupons don't exceed the budget
constraints ## all contrains

	 Restriction 2 : 22.0 ...



{1: 1*x_1_10 + 1*x_1_2 + 1*x_1_3 + 1*x_1_4 + 1*x_1_5 + 1*x_1_6 + 1*x_1_7 + 1*x_1_8 + 1*x_1_9 + -1 = 0,
 2: 1*x_2_10 + 1*x_2_2 + 1*x_2_3 + 1*x_2_4 + 1*x_2_5 + 1*x_2_6 + 1*x_2_7 + 1*x_2_8 + 1*x_2_9 + -1 = 0,
 3: 1*x_3_10 + 1*x_3_2 + 1*x_3_3 + 1*x_3_4 + 1*x_3_5 + 1*x_3_6 + 1*x_3_7 + 1*x_3_8 + 1*x_3_9 + -1 = 0,
 -1: 10*x_1_10 + 2*x_1_2 + 3*x_1_3 + 4*x_1_4 + 5*x_1_5 + 6*x_1_6 + 7*x_1_7 + 8*x_1_8 + 9*x_1_9 + 10*x_2_10 + 2*x_2_2 + 3*x_2_3 + 4*x_2_4 + 5*x_2_5 + 6*x_2_6 + 7*x_2_7 + 8*x_2_8 + 9*x_2_9 + 10*x_3_10 + 2*x_3_2 + 3*x_3_3 + 4*x_3_4 + 5*x_3_5 + 6*x_3_6 + 7*x_3_7 + 8*x_3_8 + 9*x_3_9 + -22.0 <= 0}

In [14]:
df = pd.DataFrame(columns=['coupon_optimal','p_coupon_optimal'])
objective = plp.lpSum(x_vars[i, j] * c[i, j] * d[i,j]
                      for i in set_I for j in coupons_per_contract[i])
# our model objective maximazing expected LTV - cost 
# matrix d (LTV - cost)* matrix c (prob_matrix) * matrix x (matrix of selecting only on coupon per client)

for k in constraints.keys():
    opt_model.addConstraint(constraints[k])

opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

#opt_model.solve(plp.COIN(fracGap=0.002, msg=0))
opt_model.solve(plp.PULP_CBC_CMD(fracGap=0.002, msg=0))
#opt_model.solve() ## solve the model

print('\tProblem {}!\n'.format(plp.constants.LpStatus[opt_model.status])) ## status

print('\tPrepare coupon data\n')
sel_coupon = pd.Series([])
for var in opt_model.variables():

    if var.varValue == 1:
        i, j = int(var.name.split('_')[1]), int(var.name.split('_')[2]) ##saving the optimal results
       

        sel_coupon[i] = {'AMOUNT_RC': j,
                                  'p': prob.loc[i]['prob_{}'.format(j)],
                                 }

	Problem Optimal!

	Prepare coupon data



/home/wilomaku/anaconda3/lib/python3.8/site-packages/pulp/apis/coin_api.py:71: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")
<ipython-input-14-109604fba953>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sel_coupon = pd.Series([])


In [15]:
print(plp.listSolvers())
print(plp.listSolvers(onlyAvailable=True))

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'CPLEX_DLL', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'PULP_CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD']
['PULP_CBC_CMD', 'COIN_CMD']


In [16]:
sel_coupon

1    {'AMOUNT_RC': 10, 'p': 0.8}
2    {'AMOUNT_RC': 2, 'p': 0.04}
3    {'AMOUNT_RC': 10, 'p': 0.3}
dtype: object